# Demo Test

In [3]:
from MoviePredicter import MoviePredicter
import pandas as pd
import numpy as np

In [1]:
def predict(data, zero_val=True):
    # Selects a random row and a random column from the data matrix
    i = np.random.randint(0, data.shape[0])
    j = np.random.randint(0, data.shape[1])

    real_score = data[i][j]

    if zero_val:
        while real_score == 0:
            i = np.random.randint(0, data.shape[0])
            j = np.random.randint(0, data.shape[1])

            real_score = data[i][j]

    # Inserts noise into the data matrix
    data[i][j] = np.random.randint(0, 5)

    mp = MoviePredicter(data)

    predicted_score = mp.predict(i, j)

    print(f'Real score: {real_score}')
    print(f'Predicted score: {predicted_score}')

    return real_score - predicted_score

In [7]:
df = pd.read_csv('ratings_small.csv')

data = pd.pivot_table(df, values='rating', index='userId', columns='movieId', fill_value=0) # fill_value=np.mean(df['rating'])

data = data.to_numpy()

In [8]:
errors = []
for i in range(10):
    errors.append(predict(data))

Real score: 3.543608255669773
Predicted score: 3.5392720218713944
Real score: 3.543608255669773
Predicted score: 3.4947386786457297
Real score: 3.543608255669773
Predicted score: 0.030171187988529406
Real score: 3.543608255669773
Predicted score: 3.4694867729895806
Real score: 3.543608255669773
Predicted score: 3.544094434789726
Real score: 3.543608255669773
Predicted score: 3.446398728582137
Real score: 3.543608255669773
Predicted score: 3.5350017739455266
Real score: 3.543608255669773
Predicted score: 3.5361785355097624
Real score: 3.543608255669773
Predicted score: 3.443957358409319
Real score: 3.543608255669773
Predicted score: 3.5456477534717212


In [9]:
print(errors)

[0.004336233798378775, 0.04886957702404349, 3.513437067681244, 0.07412148268019259, -0.0004861791199526344, 0.09720952708763608, 0.008606481724246606, 0.0074297201600108664, 0.0996508972604544, -0.0020394978019480092]


In [6]:
print(errors)

[4.72544355550091, 4.1996877329719755, 2.1459848574393137, 2.1868634496669577, 1.7113184167267814, 2.4122336585544364, 3.950804157649862, -0.004351741809696108, 0.27375139251438085, 1.101047131044091]


In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.hist(errors, bins=5)
plt.show()